In [86]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from tslearn.clustering import KShape
import numpy as np
from sklearn import tree

In [87]:
Restriction = '3-SD'
numberOfFeatures2 = numberOfWindows = 187
ep = 200
ep_es = 35

In [88]:
FeatureSetB1 = pd.read_csv('Windows/ptdb_187Ws.csv')
FeatureSetB1.drop("Unnamed: 0",axis=1,inplace = True)
FeatureSetB1 = FeatureSetB1.drop('index', axis=1)
tree = tree.DecisionTreeClassifier()
gbc = GradientBoostingClassifier(random_state=0)

In [89]:
FeatureSetB2 = FeatureSetB1.sample(frac=1)
FeatureSetB3 = FeatureSetB2.iloc[:, :-2]

In [90]:
import pandas as pd
import numpy as np

B_class_0 = FeatureSetB3[FeatureSetB2['Class'] == 0]
B_class_1 = FeatureSetB3[FeatureSetB2['Class'] == 1]

B_class_F0 = FeatureSetB2[FeatureSetB2['Class'] == 0]
B_class_F1 = FeatureSetB2[FeatureSetB2['Class'] == 1]


# Calculate the Z-scores for each feature
z_score_0 = (B_class_0 - B_class_0.mean()) / B_class_0.std()
z_score_1 = (B_class_1 - B_class_1.mean()) / B_class_1.std()

# Set a threshold for the Z-score
threshold_0 = 2.5
threshold_1 = 3

# Identify the outliers based on the threshold
outliers_0 = np.abs(z_score_0) > threshold_0
outliers_1 = np.abs(z_score_1) > threshold_1

# Remove the outliers from the dataset
B_class_F0 = B_class_F0[~outliers_0.any(axis=1)]
B_class_F1 = B_class_F1[~outliers_1.any(axis=1)]

# Split the dataset into two classes and perform clustering as described in the previous answer
merged_df = pd.concat([B_class_F1, B_class_F0])
merged_df = merged_df.sample(frac=1)

In [91]:
X = merged_df.iloc[:, :-2]
y = merged_df[['Class']]

In [92]:
ros = RandomOverSampler(random_state=0,sampling_strategy=0.5)

In [93]:
X_train1, X_test, y_train1, y_test = train_test_split(X,y, test_size=0.2, random_state=1, stratify=y, shuffle=True)

x,y = ros.fit_resample(X_train1,y_train1)

X_train2, X_val, y_train2, y_val = train_test_split(x,y, test_size=0.2, random_state=1, stratify=y, shuffle=True)

Rye - Artificial Neural Network 

In [94]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten, Activation
from sklearn import metrics
from keras.layers import LSTM
import tensorflow as tf
import sys
mod=sys.modules[__name__]

In [95]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn import tree

import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
tree = tree.DecisionTreeClassifier()

acc_score = []
re_score = []
pre_score = []
history_1 = []
f_score = []
auroc_score = []
auprc_score = []

In [96]:
scaler = preprocessing.Normalizer().fit(X_train2)

X_train4 = scaler.transform(X_train2)
X_test4 = scaler.transform(X_test)
X_val4 = scaler.transform(X_val)

#y_train2 = to_categorical(y_train2)
#y_test = to_categorical(y_test)
#y_val = to_categorical(y_val)

In [97]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=None,bootstrap=True,min_samples_leaf=2,min_samples_split=2,max_features='sqrt', random_state=42,n_estimators=1800)

#scaler = preprocessing.StandardScaler().fit(X_train)
#scaler = preprocessing.RobustScaler().fit(X_train)
#scaler =  preprocessing.MinMaxScaler().fit(X_train)
scaler = preprocessing.Normalizer().fit(X_train4)

X_train4 = scaler.transform(X_train4)
X_test4 = scaler.transform(X_test4)
X_val4 = scaler.transform(X_val4)

history = model.fit(X_train4,y_train2)#, epochs = ep,batch_size=512,validation_data=(X_val4, y_val), callbacks=[stop_early],shuffle=True)#,use_multiprocessing=True
history_1.append(history)

In [98]:
# Make predictions on test data
y_pred_classes = model.predict(X_test4)
y_test_classes = y_test

In [99]:
# Calculate F1 score macro and Precision macro
f1_value = f1_score(y_test_classes, y_pred_classes, average='macro')
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
sensitivity = recall_score(y_test_classes, y_pred_classes,average='macro')#,pos_label = 1, average='binary')
accuracy = accuracy_score(y_test_classes, y_pred_classes)

acc_score.append(accuracy)
re_score.append(sensitivity)
pre_score.append(precision)
f_score.append(f1_value)

k = 1 

avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k

sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score

In [100]:
#print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
#print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
#print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
#print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))

Avg accuracy : 0.9922360248447205
Avg Reccall : 0.7484326018808778
Avg Precision : 0.7976525821596244
Avg F1_score : 0.7707695593365131


In [101]:
# #scaler = preprocessing.StandardScaler().fit(X_train)
# #scaler = preprocessing.RobustScaler().fit(X_train)
# #scaler =  preprocessing.MinMaxScaler().fit(X_train)
# scaler = preprocessing.Normalizer().fit(x)

# x = scaler.transform(x)
# y = to_categorical(y)


# precision = cross_val_score(tree,x,y,cv=5,scoring='precision_macro')#,average='macro')
# recall = cross_val_score(tree,x,y,cv=5,scoring='recall_macro')#,average='macro')
# accuracy = cross_val_score(tree,x,y,cv=5,scoring='accuracy')#,average='macro')
# f1 = cross_val_score(tree,x,y,cv=5,scoring='f1_macro')#,average='macro')

# sensitivity = np.mean(recall)
# precision = np.mean(precision)
# accuracy = np.mean(accuracy)
# f1_score = np.mean(f1)  

In [102]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','NumberOfWindows','Epochs','Run_Time']
dict_data = [{'model-type':'RF', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,"Epochs":ep}]
metric_file = "Results_Apr3/ptdb_RF_1.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")